In [1]:
# -*- coding: utf-8 -*-
import requests    #此处采用requests方法得到网页响应
import json
import time
from pandas import DataFrame
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()
      
def getTagData(tag):
    data_oneTag=[]    #待添加数据列表
    page_start=0    #起始页
    data_oneSubject=[1]    #为了冷启动，使data_oneSubject不为空
      #设置代理
    proxies={ "http": "http://139.129.166.68：3128"}

   # httpproxy_handler = urllib2.ProxyHandler(proxies)
      #设置User-Agent
    headers={"User-Agent":'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'}
      #当data_oneSubject不为空，也就是始终可以从网页中获取内容时，不停循环    
    while(data_oneSubject!=[]):
          #通过修改tag和page_start来不断的修改网址
          url='https://movie.douban.com/j/search_subjects?type=movie&tag='+tag+'&sort=recommend&page_limit=20&page_start='+str(page_start)
          resp=requests.get(url,headers=headers,proxies=proxies)#,proxies=proxies
          #resp=requests.get(url,proxies=proxies,headers=headers)  #发出获取内容请求，得到的resp.text为json字符串
          data_oneSubject=json.loads(resp.text)  #将json字符串resp.text转换成Python形式，得到的data_oneSubject整体为一个键为'subjects'的长度为1的字典
          data_oneSubject=data_oneSubject['subjects']   #取出data_oneSubject字典中键'subjects'对应的值，为20个字典
          data_oneTag+=data_oneSubject    #将data_oneSubject添加到data_oneTag数据中
          page_start+=20  #起始页增加20
          time.sleep(1)    #为了避免请求的太频繁被封掉IP，所以每次循环都要暂停一秒
    return data_oneTag    #返回标签为tag时所有获得的数据
  
data_allTag=[]    #待添加所有标签的数据集列表
moviesNum=0    #所有标签下的电影总数，该数字没有消除重复项
tag='热门'     #可以添加其它列表，搞成一个列表
data_oneTag=getTagData(tag)   #针对每个标签调用getTagData函数  
data_allTag+=data_oneTag    #将每个标签下的得到的数据加入到data_allTag数据集中，最终的data_allTag为数据list数据结构，每一项为一个字典
moviesNum+=len(data_oneTag)     #计算所有标签下电影的总数（包含重复项）
print(tag+':',len(data_oneTag))    #打印出各个标签下得到的电影数
  
print('电影总数为:',moviesNum) #打印出总电影数，该数字没有消除重复项
for i in range(10):
    print(data_allTag[i]['title']+' '+data_allTag[i]['rate'])    #test
  
df=DataFrame(data_allTag)
  
  #title列为unicode编码格式，在将数据输出为csv格式时会出现编码问题，所以下面三行将title列数据转换为utf-8格式
title=df['title']
title=title.map(lambda x:x.encode('utf-8').decode()) #.decode()解析中文eval(x).decode()
df['title']=title
  
  #将结果中rate列的数字转换为float格式
rate=df['rate']   #将df的rate列取出，格式为Series
rate=rate.map(float)    #将rate列中的数值转换为float格式
df['rate']=rate    #将df中的rate列替换为float格式的rate列
  
  #将DataFrame格式的结果数据写入csv格式的文件中
df.to_csv('doubanMovies.csv',index=False,header=True)
#conn = create_engine('mysql://root:@localhost:3306/test?charset=utf8')     #存本地数据库   
#df.to_sql('film',conn)

热门: 280
电影总数为: 280
西虹市首富 6.6
我不是药神 9.0
网络谜踪 8.7
一出好戏 7.2
反贪风暴3 5.6
蚁人2：黄蜂女现身 7.4
摘金奇缘 6.4
碟中谍6：全面瓦解 8.2
修女 6.0
镰仓物语 6.9


In [2]:
print(df)

                                                 cover  cover_x  cover_y  \
0    https://img1.doubanio.com/view/photo/s_ratio_p...      679      950   
1    https://img3.doubanio.com/view/photo/s_ratio_p...     1080     1512   
2    https://img3.doubanio.com/view/photo/s_ratio_p...     1400     2075   
3    https://img3.doubanio.com/view/photo/s_ratio_p...     1080     1512   
4    https://img3.doubanio.com/view/photo/s_ratio_p...     2953     4134   
5    https://img1.doubanio.com/view/photo/s_ratio_p...     1968     2756   
6    https://img3.doubanio.com/view/photo/s_ratio_p...     2919     4325   
7    https://img3.doubanio.com/view/photo/s_ratio_p...     2048     3034   
8    https://img3.doubanio.com/view/photo/s_ratio_p...     2764     4096   
9    https://img1.doubanio.com/view/photo/s_ratio_p...     2143     2993   
10   https://img3.doubanio.com/view/photo/s_ratio_p...     1383     2048   
11   https://img1.doubanio.com/view/photo/s_ratio_p...      694     1000   
12   https:/

In [3]:
import re
#def get_imdbnumber():
url='https://movie.douban.com/subject/27605698/'
a=requests.get(url)
print(a.text)
#a=re.findall('',a.text)

<!DOCTYPE html>
<html lang="zh-cmn-Hans" class="">
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <meta name="renderer" content="webkit">
    <meta name="referrer" content="always">
    <meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />
    <title>
        西虹市首富 (豆瓣)
</title>
    
    <meta name="baidu-site-verification" content="cZdR4xxR7RxmM4zE" />
    <meta http-equiv="Pragma" content="no-cache">
    <meta http-equiv="Expires" content="Sun, 6 Mar 2005 01:00:00 GMT">
    
    <link rel="apple-touch-icon" href="https://img3.doubanio.com/f/movie/d59b2715fdea4968a450ee5f6c95c7d7a2030065/pics/movie/apple-touch-icon.png">
    <link href="https://img3.doubanio.com/f/shire/bf61b1fa02f564a4a8f809da7c7179b883a56146/css/douban.css" rel="stylesheet" type="text/css">
    <link href="https://img3.doubanio.com/f/shire/ae3f5a3e3085968370b1fc63afcecb22d3284848/css/separation/_all.css" rel="stylesheet" type="text/css">
  

In [4]:
a=re.findall('<span class="pl">IMDb链接:.*? target="_blank" rel="nofollow">(.*?)</a>',a.text)
print(a)

['tt8529186']


In [1]:
#电影标签补充信息爬取
import pandas as pd
import re
import requests
import time
proxies={ "http": "http://139.129.166.68：3128"}

   # httpproxy_handler = urllib2.ProxyHandler(proxies)
      #设置User-Agent
headers={"User-Agent":'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'}
def get_detail(url):
    detail=[]
    a=requests.get(url,headers=headers,proxies=proxies)
    detail.append(''.join(re.findall('<span class="pl">IMDb链接:.*? target="_blank" rel="nofollow">(.*?)</a>',a.text)))  #imdb号
    detail.append(' '.join(re.findall('<span ><span class=\'pl\'>导演</span>: .*>(.*?)</a>',a.text)))   #导演信息
    detail.append(' '.join(re.findall('<a href="/celebrity/.*?/" rel="v:starring">(.*?)</a>',a.text)))  #演员信息
    detail.append(' '.join(re.findall('<span property="v:genre">(.*?)</span>',a.text)))               #电影类别
    detail.append(''.join(re.findall('<span class="pl">制片国家/地区:</span>(.*?)<br/>',a.text)))      #地区
    detail.append(' '.join(re.findall('<span property="v:initialReleaseDate" content="(.*?)">',a.text)))  #上映日期，里面包含是否参加过电影节
    detail.append(''.join(re.findall('<span property="v:runtime" content="(.*?)">',a.text)))            #电影时长
    detail.append(''.join(re.findall('\u3000\u3000(.*?)\n',a.text,re.S)))                           #电影简介
    return detail
df1=pd.read_csv('doubanMovies.csv',encoding='gbk')
#df1['imdb']=None
info=[]
for i in df1.index:
    info.append(get_detail(df1['url'][i]))
    time.sleep(1)
df2=pd.DataFrame(info,columns=['imdb','direct','act','label','country','date','time','abstract'])
result=pd.concat([df1,df2],axis=1)         #与原信息合并
result.to_csv('doubanMovies.csv',index=False)     

In [37]:
'''测试信息'''
# import requests
# url='https://movie.douban.com/subject/27615441/'
# a=requests.get(url)
# a=re.findall('<span class="pl">IMDb链接:.*? target="_blank" rel="nofollow">(.*?)</a>',a.text)
# print(a.text)
# import re
# direct=' '.join(re.findall('<span ><span class=\'pl\'>导演</span>: .*>(.*?)</a>',a.text)) #导演列表
# cast=' '.join(re.findall('<a href="/celebrity/.*?/" rel="v:starring">(.*?)</a>',a.text)) #演员列表
# label=' '.join(re.findall('<span property="v:genre">(.*?)</span>',a.text))              #影片类别
# country=''.join(re.findall('<span class="pl">制片国家/地区:</span>(.*?)<br/>',a.text))  #国家
# date=' '.join(re.findall('<span property="v:initialReleaseDate" content="(.*?)">',a.text))    #上映日期里有电影节信息
# time=''.join(re.findall('<span property="v:runtime" content="(.*?)">',a.text))
# abstract=''.join(re.findall('\u3000\u3000(.*?)\n',a.text,re.S))
# print(direct)
# print(cast)
# print(label)
# print(country)
# print(date)
# print(time)
# print(abstract)

阿尼什·查甘蒂
约翰·赵 米切尔·拉 黛博拉·梅辛 约瑟夫·李 萨拉·米博·孙 亚历克丝·杰恩·高 梅金·刘 刘卡雅 多米尼克·霍夫曼 西尔维亚·米纳西安 梅丽莎·迪斯尼 康纳·麦克雷斯 科林·伍德尔 约瑟夫·约翰·谢尔勒 阿什丽·艾德纳 托马斯·巴布萨卡 朱莉·内桑森 罗伊·阿布拉姆森 盖奇·比尔托福 肖恩·奥布赖恩 瑞克·萨拉比亚 布拉德·阿布瑞尔 加布里埃尔D·安吉尔
剧情 悬疑 惊悚
 美国 / 俄罗斯
2018-01-20(圣丹斯电影节) 2018-08-24(美国)
102
故事发生在北加州的圣荷西，玛格（米切尔·拉 Michelle La 饰）成长在一个幸福的家庭之中，父亲金（约翰·赵 John Cho 饰）和母亲帕尔马（萨拉·米博·索恩 Sara Mibo Sohn 饰）非常的恩爱，他们用摄像机记录了玛格成长的一点一滴，在女儿的身上，两人可谓倾注了全部的心血。然而，不幸就这样降临了，帕尔马患上了癌症，虽然全家人合力同病魔抗争，但最终帕尔马还是未能战胜病魔。母亲去世了，全家的重担都落在了金的身上，坚强的他独自将女儿抚养长大，虽然玛格非常的听话懂事，但随着时间的推移，金还是渐渐感受到了和女儿之间的隔阂。一天，玛格在放学后没有按时回家，直到第二天早晨也毫无音信。


In [1]:
#信息分析code
import pandas as pd
from collections import Counter
from pyecharts import Pie
from pyecharts import Map
df = pd.read_csv('doubanMovies.csv',encoding='gbk')
def coun(s):
    return s.replace(' ','').split('/')  #多国家格式有‘/’隔开，需要将其转化为列表
#电影类别分析（总，按评分，地区）
def to_list(s):
    return s.split()      #类别标签以空格形式储存需要转化为列表
label = df['label'].apply(to_list).tolist()
label = Counter(sum(label,[]))
label_sort = sorted(label.items(),key = lambda x:x[1],reverse=True)
#print(label_sort)
label_sort = pd.DataFrame(label_sort,columns=['label','num'])
label = label_sort['label'].tolist()
num = label_sort['num'].tolist()
pie = Pie("热门电影类别",title_pos='right')
pie.add("",label,num,center=[60,60],is_label_show=True,legend_orient="vertical", legend_pos="left")
pie
#-----------------------------------------


ERROR:lml.utils:failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\lml\utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


评分top10：
9	我不是药神
8.7	网络谜踪
8.7	幸福的拉扎罗
8.7	头号玩家
8.7	机动战士高达THE ORIGIN VI 赤色彗星诞生
8.6	奇迹男孩
8.4	完美陌生人(西班牙版)
8.4	中二病也要谈恋爱！剧场版 -Take On Me-
8.3	特工
8.3	沉默的教室


In [2]:
#结合评分的类别信息
df1=df[df['rate']>=7]
label = df1 ['label'].apply(to_list).tolist()
label = Counter(sum(label,[]))
label_sort = sorted(label.items(),key = lambda x:x[1],reverse=True)
#print(label_sort)
label_sort = pd.DataFrame(label_sort,columns=['label','num'])
label = label_sort['label'].tolist()
num = label_sort['num'].tolist()
pie = Pie("7+电影类别",title_pos='right')
pie.add("",label,num,center=[60,60],is_label_show=True,legend_orient="vertical", legend_pos="left")
pie


In [4]:
df1=df[(df['rate']>=5) & (df['rate']<7) ]
label = df1 ['label'].apply(to_list).tolist()
label = Counter(sum(label,[]))
label_sort = sorted(label.items(),key = lambda x:x[1],reverse=True)
label_sort = pd.DataFrame(label_sort,columns=['label','num'])
label = label_sort['label'].tolist()
num = label_sort['num'].tolist()
pie = Pie("5-7电影类别",title_pos='right')
pie.add("",label,num,center=[60,60],is_label_show=True,legend_orient="vertical", legend_pos="left")
pie

In [5]:
df1=df[(df['rate']>=3) & (df['rate']<5) ]
label = df1 ['label'].apply(to_list).tolist()
label = Counter(sum(label,[]))
label_sort = sorted(label.items(),key = lambda x:x[1],reverse=True)
label_sort = pd.DataFrame(label_sort,columns=['label','num'])
label = label_sort['label'].tolist()
num = label_sort['num'].tolist()
pie = Pie("3-5电影类别",title_pos='right')
pie.add("",label,num,center=[60,60],is_label_show=True,legend_orient="vertical", legend_pos="left")
pie

剧情占比变少，说明以上类别电影低分的主要原因是剧情太差，没营养。。可以进一步分析一下高分电影中有重复类别中的剧情占比

In [14]:
df1=df[df['rate']>=7]
df1['country']=df1['country'].apply(coun)
country = Counter(sum(df1['country'].tolist(),[]))
country = sorted(country.items(),key = lambda x:x[1],reverse=True)
name=['country_c','country_en']
country_en = pd.read_table('test.txt',sep='\t',header= None,names=name)
country_en = country_en.set_index('country_c').T.to_dict('list')          #导入地名中英文词典
country = pd.DataFrame(country,columns=['country_c','freq'])
for i in country.index:
    country['country_c'][i] =''.join(country_en[country['country_c'][i]]) 
value = country['freq'].tolist()
attr= country['country_c'].tolist()
map = Map("7+评分世界地图示例", width=1200, height=600)
map.add(
    "",
    attr,
    value,
    maptype="world",
    is_visualmap=True,
    visual_text_color="#000",
    is_map_symbol_show=False,
)
map

D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\anaconda\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [11]:
df1=df[(df['rate']>=5) & (df['rate']<7) ]
df1['country']=df1['country'].apply(coun)
country = Counter(sum(df1['country'].tolist(),[]))
country = sorted(country.items(),key = lambda x:x[1],reverse=True)
name=['country_c','country_en']
country_en = pd.read_table('test.txt',sep='\t',header= None,names=name)
country_en = country_en.set_index('country_c').T.to_dict('list')          #导入地名中英文词典
country = pd.DataFrame(country,columns=['country_c','freq'])
for i in country.index:
    country['country_c'][i] =''.join(country_en[country['country_c'][i]]) 
value = country['freq'].tolist()
attr= country['country_c'].tolist()
map = Map("5-7评分世界地图示例", width=1200, height=600)
map.add(
    "",
    attr,
    value,
    maptype="world",
    is_visualmap=True,
    visual_text_color="#000",
    is_map_symbol_show=False,
)
map

D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\anaconda\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [13]:
df1=df[(df['rate']>=3) & (df['rate']<5) ]
df1['country']=df1['country'].apply(coun)
country = Counter(sum(df1['country'].tolist(),[]))
country = sorted(country.items(),key = lambda x:x[1],reverse=True)
name=['country_c','country_en']
country_en = pd.read_table('test.txt',sep='\t',header= None,names=name)
country_en = country_en.set_index('country_c').T.to_dict('list')          #导入地名中英文词典
country = pd.DataFrame(country,columns=['country_c','freq'])
for i in country.index:
    country['country_c'][i] =''.join(country_en[country['country_c'][i]]) 
value = country['freq'].tolist()
attr= country['country_c'].tolist()
map = Map("3-5评分世界地图示例", width=1200, height=600)
map.add(
    "",
    attr,
    value,
    maptype="world",
    is_visualmap=True,
    visual_text_color="#000",
    is_map_symbol_show=False,
)
map

D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\anaconda\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [37]:
#地区分析
df['country']=df['country'].apply(coun)
country = Counter(sum(df['country'].tolist(),[]))


In [41]:
country = sorted(country.items(),key = lambda x:x[1],reverse=True)
name=['country_c','country_en']
country_en = pd.read_table('test.txt',sep='\t',header= None,names=name)
country_en = country_en.set_index('country_c').T.to_dict('list')          #导入地名中英文词典


In [42]:
country = pd.DataFrame(country,columns=['country_c','freq'])
for i in country.index:
    country['country_c'][i] =''.join(country_en[country['country_c'][i]]) 

D:\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,country_c,freq
0,United States,110
1,China,68
2,Japan,33
3,Korea,29
4,France,21
5,United Kingdom,20
6,China,18
7,Germany,8
8,Canada,7
9,India,6


In [64]:
from pyecharts import Boxplot
import numpy as np
def remove(a):
    while '0' in a:
        a.remove('0')
    return a                   
df.fillna('0',inplace=True)                   #为去除nan做准备
x_axis = ['all','7+','5-7','3-5']
x_axis1 = ['8+','7-8','6-7','5-6','5-']
time = []
time1 = []
time.append(remove(df['time'].tolist()))
df1=df[df['rate']>=7]
time.append(remove(df1['time'].tolist()))
df1=df[(df['rate']>=5) & (df['rate']<7) ]
time.append(remove(df1['time'].tolist()))
df1=df[(df['rate']>=3) & (df['rate']<5) ]
time.append(remove(df1['time'].tolist()))
boxplot = Boxplot("箱形图")
boxplot.add("粗略分布", x_axis, boxplot.prepare_data(time))


In [65]:
#-------细----------------------------------
df1=df[df['rate']>=8]
time1.append(remove(df1['time'].tolist()))
df1=df[(df['rate']>=7) & (df['rate']<8) ]
time1.append(remove(df1['time'].tolist()))
df1=df[(df['rate']>=6) & (df['rate']<7) ]
time1.append(remove(df1['time'].tolist()))
df1=df[(df['rate']>=5) & (df['rate']<6) ]
time1.append(remove(df1['time'].tolist()))
df1=df[df['rate']<5]
time1.append(remove(df1['time'].tolist()))
boxplot1 = Boxplot("箱形图")
boxplot1.add("细分布", x_axis1, boxplot.prepare_data(time1))
boxplot1


In [3]:
df['abstract']

0      西虹市丙级球队大翔队的守门员王多鱼（沈腾 饰）因比赛失利被教练开除，一筹莫展之际王多鱼突然收...
1      普通中年男子程勇（徐峥 饰）经营着一家保健品店，失意又失婚。不速之客吕受益（王传君 饰）的到...
2      故事发生在北加州的圣荷西，玛格（米切尔·拉 Michelle La 饰）成长在一个幸福的家庭...
3      马进欠下债务，与远房表弟小兴在底层社会摸爬滚打，习惯性的买彩票，企图一夜暴富，并迎娶自己的同...
4      ICAC (廉政公署) 陆志廉（古天乐 饰），JFIU (联合财富情报组) 刘保强（张智霖 ...
5      来自漫威电影宇宙的新片《蚁人2：黄蜂女现身》重点展现了两位超级英雄令人惊叹的收缩能力。继《美...
6      纽约土生土长的朱丽秋（吴恬敏 饰）陪拍拖已久的男友杨力（亨利·戈尔丁 饰）回到新加坡参加死党...
7      有时好意会造成恶果，人反而被自己所造成的结果所困扰。伊桑·亨特（汤姆·克鲁斯）和他的IMF团...
8      一个年轻修女在罗马尼亚与世隔绝的修道院自杀，梵蒂冈派去一名神父和一名快结束见习期的修女展开调...
9      镰仓，一个看似不起眼的小城，却是连通两个异世界的神奇之处，流传着各种奇妙的 传说。推理作家一...
10     《克里斯托弗·罗宾》真人版讲述了一个温馨的故事。影片中，曾在百亩森林中与这群生机勃勃、惹人怜...
11     马迪亚斯（科林·伍德尔 Colin Woodell 饰）的女友阿玛雅（斯蒂芬妮·诺格拉斯 S...
12     狮王（郑伊健 饰）、火山（陈小春 饰）、Bill（谢天华 饰）、淡定（钱嘉乐 饰）、老鼠（林...
13     德古拉和家人朋友们踏上了一艘邮轮，宿命之敌“吸血鬼猎人”范海辛正在船上布置天罗地网等待“猎物...
14     超人家族时隔14年强势回归！这次站在聚光灯下的是弹力女超人海伦（霍利·亨特 配音），超能先生...
15     在游戏机厅做着兼职“小丑”的郑开司（李易峰 饰），幼时父亲突然失踪，母亲重病住院，使得郑开司...
16     故事开始于2001年的山西大同，巧巧（赵涛 饰）和斌斌（廖凡 饰）相恋多年，巧巧一心希望能够...
17     在香港市中心，世界上最高、结构最复杂的摩天大楼遭

In [43]:
value = country['freq'].tolist()
attr= country['country_c'].tolist()
map = Map("世界地图示例", width=1200, height=600)
map.add(
    "",
    attr,
    value,
    maptype="world",
    is_visualmap=True,
    visual_text_color="#000",
    is_map_symbol_show=False,
)
map

In [14]:
#--写入国家信息，翻译英语
# country = list(set(sum(df['country'].tolist(),[])))
# f = open('test.txt', 'w',encoding = 'utf-8')
# for i in country:
#     f.write(i+'\n')
    
# f.close()


[('美国', 110),
 ('中国大陆', 68),
 ('日本', 33),
 ('韩国', 29),
 ('法国', 21),
 ('英国', 20),
 ('香港', 18),
 ('德国', 8),
 ('加拿大', 7),
 ('印度', 6),
 ('意大利', 5),
 ('俄罗斯', 4),
 ('瑞士', 3),
 ('挪威', 3),
 ('西班牙', 3),
 ('比利时', 3),
 ('波兰', 2),
 ('澳大利亚', 2),
 ('伊朗', 2),
 ('台湾', 2),
 ('丹麦', 1),
 ('墨西哥', 1),
 ('芬兰', 1),
 ('冰岛', 1),
 ('印度尼西亚', 1),
 ('马来西亚', 1),
 ('爱尔兰', 1),
 ('卢森堡', 1),
 ('荷兰', 1),
 ('奥地利', 1)]